# Making Bounding Box

In [ ]:
import cv2
from matplotlib import pyplot as plt

def draw_and_show_bounding_box(image_path, box_coordinates):
    # Load the image
    image = cv2.imread(image_path)
    print(image.shape)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    # hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    # Draw the bounding box
    x, y, width, height = box_coordinates
    roi = image[y:y+height, x:x+width]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    # Convert ROI from BGR to RGB (matplotlib displays in RGB)
    # roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    plt.imshow(roi)
    plt.axis('on')  # Hide the axis
    plt.show()
    # Use matplotlib to display the image
    plt.imshow(image)
    plt.show()
    plt.imshow(hsv_roi)
    plt.show()

# Usage
image_path = r"401055cam_1 00--03/frame_1887.jpg"
box_coordinates = (480, 74, 10, 10)
#box_coordinates = (1758, 148, 12, 12)
draw_and_show_bounding_box(image_path, box_coordinates)

# Finding Frames of the particular hour.

In [2]:
import cv2
import os

def extract_frames(video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    print(3)
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    frame_count = 0
    num = frame_count
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if (frame_count>=0):

            # Write the frame to the output folder
            frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
        if (frame_count>=108000):
            print(4)
            break
    print(5)
    cap.release()
    frameExtracted = frame_count - num
    print(f"Total frames extracted: {frameExtracted} ")

# Example usage
path=r"theVideos/close"
videos=os.listdir(path)
i=1
print(1.0)
for video in videos:
    video_path = os.path.join(path,video)
    output_folder=video.split('_')[0]+f'cam_{i}'+' 00--03'
    print(2)
    extract_frames(video_path, output_folder)
    
    i+=1



1.0
2
3
4
5
Total frames extracted: 108000 


# Making hour long videos from those frames.

In [4]:
import cv2
import os
from natsort import natsorted

# Path to the directory containing images
image_folder = 'hourlyFrame/mid/mid 400658cam_2 00--01'

# Path to the directory where the video will be saved
output_folder = 'hourlyVideos/midCam'

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Get the list of all files in the directory
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sort the images using natsorted to ensure they are in natural order
images = natsorted(images)

# Read the first image to get the dimensions
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Define the codec and create a VideoWriter object
video_name = 'mid 00--01.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 10  # Frames per second
video_path = os.path.join(output_folder, video_name)
video = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Iterate through all images and write them to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the VideoWriter object
video.release()

print(f'Video {video_name} created successfully at {video_path}.')


Video mid 00--01.mp4 created successfully at hourlyVideos/midCam/mid 00--01.mp4.


# Combining the three one hour videos to create synced video

In [6]:
import cv2
import numpy as np

# Function to resize a frame to a specific height while maintaining aspect ratio
def resize_frame(frame, height):
    (h, w) = frame.shape[:2]
    ratio = height / float(h)
    dimension = (int(w * ratio), height)
    return cv2.resize(frame, dimension, interpolation=cv2.INTER_AREA)

# Load the video files
cap1 = cv2.VideoCapture('hourlyVideos/farCam/far 00--01.mp4')
cap2 = cv2.VideoCapture('hourlyVideos/midCam/mid 00--01.mp4')
cap3 = cv2.VideoCapture('hourlyVideos/closeCam/close 00--01.mp4')

# Get properties from the first video
fps = int(cap1.get(cv2.CAP_PROP_FPS))
width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('syncedVideo_00--01.mp4', fourcc, fps, (width * 3, height))

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()

    # Break the loop if any of the videos ends
    if not ret1 or not ret2 or not ret3:
        break

    # Resize frames to the same height
    frame2 = resize_frame(frame2, height)
    frame3 = resize_frame(frame3, height)

    # Concatenate frames horizontally
    combined_frame = np.hstack((frame1, frame2, frame3))

    # Write the combined frame to the output video
    out.write(combined_frame)

# Release everything when done
cap1.release()
cap2.release()
cap3.release()
out.release()
cv2.destroyAllWindows()


# Making frames for the synced video

In [7]:
import cv2
import os

def extract_frames(video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    frame_count = 0
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        frame_count+=1
        # Write the frame to the output folder
        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)


    cap.release()
    print(f"Total frames extracted: {frame_count}")

# directory/path of the stiched video
video_path=r"syncedVideo_00--01.mp4"
output_folder='syncedFrame/00--01_hr_frames_stichedvideo'
extract_frames(video_path, output_folder)

Total frames extracted: 36000


# Finding yellow light frame periods for the closest camera.

In [2]:
%pip install numba

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 9.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
 
def draw_and_show_bounding_box(image_path, box_coordinates):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    # Draw the bounding box
    x, y, width, height = box_coordinates
    roi = image[y:y+height, x:x+width]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_RGB2HSV)  # Convert ROI to HSV
    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    # Check if the ROI contains yellow
    is_yellow_present = check_for_yellow(hsv_roi)
    return is_yellow_present, image
 
def check_for_yellow(hsv_image):
    # Define the range for yellow color in HSV
    lower_yellow = np.array([21, 13, 230])
    upper_yellow = np.array([42,77, 255])
    # Create a mask to extract yellow areas
    mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    # Check if there are any yellow pixels in the ROI
    yellow_pixels = cv2.countNonZero(mask)
    return yellow_pixels > 0.2
 
def process_frames_in_folder(folder_path, box_coordinates, merge_threshold=10):
    yellow_frames = []  # List to store frame numbers with yellow detected
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            is_yellow_present, _ = draw_and_show_bounding_box(image_path, box_coordinates)
            frame_number = int(os.path.splitext(filename)[0].split('_')[-1])  # Assuming frame number is in the filename
            if is_yellow_present:
                yellow_frames.append(frame_number)
    
    yellow_frames.sort()  # Sort frame numbers in ascending order
    
    # Initialize variables to track ranges
    ranges = []
    start = yellow_frames[0]
    end = yellow_frames[0]
    
    # Iterate through sorted yellow_frames to find ranges
    for i in range(1, len(yellow_frames)):
        if yellow_frames[i] <= end + merge_threshold:
            end = yellow_frames[i]
        else:
            ranges.append((start, end))
            start = yellow_frames[i]
            end = yellow_frames[i]
    
    # Append the last range
    ranges.append((start, end))
    
    print(ranges)
 
folder_path = "401055cam_1 00--03"
box_coordinates = (480, 74, 10, 10)  # Adjust these coordinates as per your requirement
 
process_frames_in_folder(folder_path, box_coordinates)

Corrupt JPEG data: 426 extraneous bytes before marker 0xd9


[(4, 78), (122, 122), (201, 300), (339, 350), (548, 554), (722, 723), (852, 857), (1081, 1110), (1191, 1240), (1330, 1381), (1442, 1443), (1561, 1561), (1681, 1681), (1696, 1698), (1747, 1747), (1801, 1801), (1881, 1932), (1948, 1991), (2013, 2040), (2067, 2067), (2081, 2083), (2102, 2104), (2149, 2159), (2526, 2532), (2581, 2581), (2594, 2596), (2623, 2625), (2721, 2821), (3023, 3024), (3199, 3282), (3301, 3306), (3659, 3716), (3962, 3962), (4023, 4031), (4214, 4294), (4349, 4383), (4394, 4397), (4564, 4611), (4681, 4681), (4747, 4758), (4775, 4786), (4923, 4924), (4996, 5006), (5027, 5027), (5041, 5041), (5101, 5112), (5158, 5162), (5204, 5223), (5336, 5398), (5461, 5473), (5521, 5550), (5568, 5580), (5651, 5652), (5670, 5704), (5730, 5730), (5770, 5772), (5824, 5832), (5881, 5882), (6185, 6186), (6241, 6269), (6869, 6917), (6931, 6931), (7201, 7205), (7464, 7501), (7562, 7572), (7585, 7586), (7608, 7623), (7748, 7748), (7981, 7981), (8174, 8274), (8312, 8332), (8470, 8473), (8550, 8

# Making 20 second yellow light clips

In [17]:
import cv2
import os

# Define the directory containing the frame images
frame_directory = 'syncedFrame/05--06_hr_frames_stichedvideo'

# Define the yellow frames intervals
the_yellow_frames = [(180858, 180911), (181283, 181337), (181956, 182009), (182652, 182705), (183051, 183105), (183519, 183573), (183994, 184047), (184840, 184893), (185349, 185403), (186914, 186968), (187872, 187925), (188289, 188342), (189362, 189416), (189855, 189909), (190275, 190316), (190320, 190322), (190325, 190325), (190855, 190908), (191270, 191323), (191688, 191741), (192174, 192227), (192577, 192630), (193176, 193230), (193719, 193772), (194788, 194842), (195245, 195299), (195702, 195755), (197048, 197102), (198022, 198038), (198040, 198045), (198047, 198049), (198051, 198052), (198054, 198058), (198060, 198061), (198063, 198064), (198067, 198076), (198946, 199000), (199423, 199476), (199926, 199980), (200412, 200466), (200843, 200897), (201924, 201941), (201943, 201948), (201950, 201952), (201954, 201955), (201957, 201958), (201966, 201966), (202789, 202843), (204461, 204515), (205523, 205577), (206248, 206283), (206286, 206286), (206288, 206297), (206299, 206302), (206749, 206802), (207473, 207526), (207938, 207992), (208481, 208534), (209084, 209137), (210123, 210176), (210776, 210829), (211603, 211657), (212769, 212774), (212777, 212778), (212781, 212822), (213341, 213395), (213806, 213859), (214030, 214030), (214047, 214047), (214057, 214060), (214062, 214062), (214711, 214764), (214784, 214784), (214798, 214798), (214808, 214809), (214893, 214894), (214903, 214905), (214926, 214926), (214930, 214931), (214955, 214955), (214962, 214963), (215006, 215007), (215009, 215009), (215011, 215024), (215026, 215032), (215181, 215182), (215184, 215184), (215186, 215189), (215191, 215191), (215193, 215193), (215195, 215195), (215198, 215198), (215200, 215201), (215203, 215203), (215205, 215206), (215208, 215209), (215215, 215215), (215217, 215219), (215221, 215224), (215226, 215233), (215239, 215243), (215245, 215246), (215248, 215249), (215251, 215251), (215253, 215254), (215256, 215256), (215260, 215261), (215263, 215264), (215267, 215268), (215270, 215270), (215745, 215747), (215749, 215799)]

print(len(the_yellow_frames))
# Function to create a video clip from a sequence of frames
def create_video_clip(start_frame, end_frame, frame_dir, output_dir, fps=10):
    frame_files = []
    
    for i in range(start_frame, end_frame + 1):
        frame_file = os.path.join(frame_dir, f"frame_{i}.jpg")
        if os.path.exists(frame_file):
            frame_files.append(frame_file)
        else:
            print(f"Frame {i} does not exist, skipping...")
            continue
    
    if not frame_files:
        print(f"No frames found for the interval {start_frame} to {end_frame}. Skipping...")
        return
    
    # Read the first frame to get the frame size
    frame = cv2.imread(frame_files[0])
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
    output_file = os.path.join(output_dir, f"clip_{start_frame}_{end_frame}.mp4")
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        video.write(frame)

    video.release()

# Directory to save the generated video clips
output_directory = 'yellowLight/yellow 05--06'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop through each tuple in the_yellow_frames and create a video clip
for start, end in the_yellow_frames:
    start = start - 80
    end = end + 80
    create_video_clip(start, end, frame_directory, output_directory)

print("Video clips have been created successfully.")


112


KeyboardInterrupt: 